In [1]:
# Design of the network.

# Settings and packages

from __future__ import division
import time
import sys
import math
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import rc
import numpy as np
from keras.layers import Dense, Input, Activation, Layer, Flatten, Reshape
from sklearn.model_selection import train_test_split
from keras.models import Sequential
import tensorflow as tf
import keras
import pandas as pd
import numpy as np
from keras.constraints import Constraint
from keras import backend as K
from tqdm import tqdm



font = {'family' : 'normal',
        'size'   : 22}

matplotlib.rc('font', **font)

# Some global constants
num_epochs = 300
batch_sz = 512

n_TA_train = 7
TA_train = [0,2,4,6,8,10,12]
TA_test = [0,1,2,3,4,5,6,7,8,9,10,11,12]
n_TA_test = 13

Nparticles = 625
Ndistances = 20
rho = 1

ngaussians = 30
std = 0.05
cutoff = 3
gaussianmeans = np.linspace(0, cutoff, ngaussians)
gaussianmeans = tf.convert_to_tensor(value=gaussianmeans, dtype='float32')


@tf.function
def map_Y_to_means(y,means):
    i = 0
    dist = tf.math.reduce_max((means-y)**2)
    for j in range(len(means)):
        if (y-means[j])**2 < dist:
            i = j
            dist = (y-means[j])**2
    return means[i]


# Design of the network.

class PolarContConv(Layer):
    def __init__(self,  npolars):
        super(PolarContConv, self).__init__()
        self.npolars = npolars

    def polarharmonics(self, coord, m):
        theta = tf.gather(coord, list(range(Nparticles, 2*Nparticles)), axis=-2)
        return m%2*tf.math.sin(theta*m) + (1-m%2)*tf.math.cos(theta*m)

    def Gaussianbasis(self, coord, kernel, m):
        x = tf.gather(coord, list(range(Nparticles)), axis=-2)
        rep=[]
        for i in range(ngaussians):
            rep.append(kernel[m][i]*tf.math.exp(-(x-gaussianmeans[i])**2/(2*std**2)))
        rep_sum = tf.math.add_n(rep)
        return rep_sum

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel',
                                      shape=(self.npolars,ngaussians),
                                      initializer='uniform',
                                      trainable=True)
        self.shape = input_shape
        super(PolarContConv, self).build(input_shape)  # Be sure to call this at the end

    def call(self, coord):
        rep=[]
        for m in range(self.npolars):
            rep.append(tf.multiply(self.Gaussianbasis(coord, self.kernel, m),self.polarharmonics(coord, m)))
        rep_concatenated =  tf.concat(rep, axis=1)
        rep_concatenated_sum_rows = tf.math.reduce_sum(rep_concatenated, axis=2, keepdims=False, name=None)
        return rep_concatenated_sum_rows

    def compute_output_shape(self, input_shape):
        return (input_shape[0], int(input_shape[1]/2)*self.npolars)
    
class EqualityConstraint(Constraint):
    def __init__(self,blah):
        self.blah = blah

    def __call__(self, w):
        average_w = tf.reduce_mean(w)
        new_w = tf.reshape(tf.repeat(average_w, w.shape[0]), w.shape)
        return new_w
    
class gen_random_var(Layer):
    def call(self, mus):
        return mus 

def RelativeEntropy(yTrue,yPred):
    result = 0
    for i in range(n_TA_train):
        means_local = tf.reshape(tf.map_fn(lambda x: map_Y_to_means(x, means_train), yTrue), tf.shape(yTrue))
        boolie = tf.equal(means_local, means_train[i])
        yTrue_local = tf.gather_nd(yTrue, tf.where(boolie))
        yPred_local = tf.gather_nd(yPred, tf.where(boolie))
        result+=K.square(K.mean(yTrue_local-yPred_local,axis=-1))
    return result

# Building the data sets

means_train = np.zeros(n_TA_train)

n = 5000;

X = np.zeros((n*n_TA_train, Nparticles*2, Ndistances))
Y = np.zeros(n*n_TA_train)

for i in range(n_TA_train):
    X_local = np.reshape(np.load('../X_TA' + str(TA_train[i]) + '.0.npy'), (6000,2*Nparticles,Ndistances))
    Y_local = np.reshape(np.load('../Y_TA'  + str(TA_train[i]) + '.0.npy'), 6000)
    
    Y_local = Y_local + 12.91*0.55
    means_train[i] = np.mean(Y_local)
    
    X[i*n:(i+1)*n, :, :] = X_local[1000:,:,:]
    Y[i*n:(i+1)*n] = Y_local[1000:]
    
rescale = max(Y) - min(Y)

Y = Y/rescale

means_train = means_train/rescale

means_train = tf.convert_to_tensor(means_train, dtype=tf.float32)

# Creating train / validation 

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=0.2)

# Creating test data

means_test = np.zeros(n_TA_test)

n = 1000;

X_test = np.zeros((n*n_TA_test, Nparticles*2, Ndistances))
Y_test = np.zeros(n*n_TA_test)

for i in range(n_TA_test):
    X_local = np.reshape(np.load('../X_TA' + str(TA_test[i]) + '.0.npy'), (6000,2*Nparticles,Ndistances))
    Y_local = np.reshape(np.load('../Y_TA'  + str(TA_test[i]) + '.0.npy'), 6000)
    
    Y_local = Y_local + 12.91*0.55
    means_test[i] = np.mean(Y_local)
    Y_output = np.ones(len(Y_local))*means_test[i]
    
    X_test[i*n:(i+1)*n, :, :] = X_local[:n,:,:]
    Y_test[i*n:(i+1)*n] = Y_output[:n]
    
Y_test = Y_test/rescale

means_test = means_test/rescale


# Constructing and running the model

model = Sequential()
model.add(PolarContConv(1))
model.add(Dense(1, kernel_constraint= EqualityConstraint(1)))
model.compile(loss=RelativeEntropy, optimizer='adam', metrics=['mse'])

history = model.fit(X_train, Y_train, epochs=num_epochs, batch_size=batch_sz, validation_data=(X_validation, Y_validation), verbose=2)

Y_pred_test = model.predict(X_test)
weights = model.get_weights()
val_los = history.history['val_loss']
los = history.history['loss']

np.save('X_test.npy', X_test)
np.save('Y_pred_test.npy', Y_pred_test)
np.save('weights_Fr.npy', weights[0][0])
np.save('weights_w.npy', weights[1])
np.save('weights_b.npy', weights[2])
np.save('val_loss.npy', val_los)
np.save('loss.npy', los)
np.save('rescale.npy', rescale)




Epoch 1/300
55/55 - 43s - loss: 1.0167 - mse: 0.1489 - val_loss: 0.3285 - val_mse: 0.0496
Epoch 2/300
55/55 - 41s - loss: 0.3077 - mse: 0.0471 - val_loss: 0.2844 - val_mse: 0.0441
Epoch 3/300
55/55 - 41s - loss: 0.2532 - mse: 0.0405 - val_loss: 0.2205 - val_mse: 0.0368
Epoch 4/300
55/55 - 41s - loss: 0.1781 - mse: 0.0329 - val_loss: 0.1351 - val_mse: 0.0296
Epoch 5/300
55/55 - 41s - loss: 0.0945 - mse: 0.0281 - val_loss: 0.0610 - val_mse: 0.0289
Epoch 6/300
55/55 - 40s - loss: 0.0451 - mse: 0.0315 - val_loss: 0.0347 - val_mse: 0.0352
Epoch 7/300
55/55 - 40s - loss: 0.0317 - mse: 0.0371 - val_loss: 0.0323 - val_mse: 0.0394
Epoch 8/300
55/55 - 40s - loss: 0.0332 - mse: 0.0395 - val_loss: 0.0333 - val_mse: 0.0401
Epoch 9/300
55/55 - 41s - loss: 0.0312 - mse: 0.0395 - val_loss: 0.0308 - val_mse: 0.0402
Epoch 10/300
55/55 - 41s - loss: 0.0316 - mse: 0.0394 - val_loss: 0.0388 - val_mse: 0.0411
Epoch 11/300
55/55 - 41s - loss: 0.0300 - mse: 0.0391 - val_loss: 0.0282 - val_mse: 0.0391
Epoch 12

55/55 - 41s - loss: 0.0153 - mse: 0.0330 - val_loss: 0.0195 - val_mse: 0.0329
Epoch 92/300
55/55 - 41s - loss: 0.0137 - mse: 0.0328 - val_loss: 0.0101 - val_mse: 0.0322
Epoch 93/300
55/55 - 41s - loss: 0.0099 - mse: 0.0327 - val_loss: 0.0082 - val_mse: 0.0319
Epoch 94/300
55/55 - 41s - loss: 0.0116 - mse: 0.0327 - val_loss: 0.0138 - val_mse: 0.0331
Epoch 95/300
55/55 - 40s - loss: 0.0098 - mse: 0.0328 - val_loss: 0.0095 - val_mse: 0.0328
Epoch 96/300
55/55 - 41s - loss: 0.0119 - mse: 0.0332 - val_loss: 0.0101 - val_mse: 0.0324
Epoch 97/300
55/55 - 40s - loss: 0.0126 - mse: 0.0333 - val_loss: 0.0152 - val_mse: 0.0333
Epoch 98/300
55/55 - 40s - loss: 0.0097 - mse: 0.0329 - val_loss: 0.0082 - val_mse: 0.0324
Epoch 99/300
55/55 - 40s - loss: 0.0154 - mse: 0.0336 - val_loss: 0.0269 - val_mse: 0.0344
Epoch 100/300
55/55 - 41s - loss: 0.0157 - mse: 0.0331 - val_loss: 0.0080 - val_mse: 0.0323
Epoch 101/300
55/55 - 40s - loss: 0.0114 - mse: 0.0331 - val_loss: 0.0178 - val_mse: 0.0336
Epoch 102/

Epoch 181/300
55/55 - 40s - loss: 0.0062 - mse: 0.0310 - val_loss: 0.0059 - val_mse: 0.0309
Epoch 182/300
55/55 - 40s - loss: 0.0097 - mse: 0.0314 - val_loss: 0.0055 - val_mse: 0.0303
Epoch 183/300
55/55 - 41s - loss: 0.0103 - mse: 0.0311 - val_loss: 0.0058 - val_mse: 0.0306
Epoch 184/300
55/55 - 40s - loss: 0.0076 - mse: 0.0311 - val_loss: 0.0120 - val_mse: 0.0314
Epoch 185/300
55/55 - 41s - loss: 0.0124 - mse: 0.0311 - val_loss: 0.0074 - val_mse: 0.0307
Epoch 186/300
55/55 - 40s - loss: 0.0095 - mse: 0.0312 - val_loss: 0.0076 - val_mse: 0.0305
Epoch 187/300
55/55 - 40s - loss: 0.0091 - mse: 0.0308 - val_loss: 0.0057 - val_mse: 0.0304
Epoch 188/300
55/55 - 41s - loss: 0.0118 - mse: 0.0311 - val_loss: 0.0190 - val_mse: 0.0316
Epoch 189/300
55/55 - 41s - loss: 0.0138 - mse: 0.0311 - val_loss: 0.0144 - val_mse: 0.0305
Epoch 190/300
55/55 - 40s - loss: 0.0089 - mse: 0.0304 - val_loss: 0.0111 - val_mse: 0.0304
Epoch 191/300
55/55 - 40s - loss: 0.0088 - mse: 0.0307 - val_loss: 0.0147 - val_

55/55 - 40s - loss: 0.0114 - mse: 0.0286 - val_loss: 0.0050 - val_mse: 0.0273
Epoch 271/300
55/55 - 41s - loss: 0.0101 - mse: 0.0284 - val_loss: 0.0061 - val_mse: 0.0273
Epoch 272/300
55/55 - 40s - loss: 0.0105 - mse: 0.0282 - val_loss: 0.0184 - val_mse: 0.0292
Epoch 273/300
55/55 - 40s - loss: 0.0088 - mse: 0.0281 - val_loss: 0.0050 - val_mse: 0.0278
Epoch 274/300
55/55 - 40s - loss: 0.0120 - mse: 0.0283 - val_loss: 0.0050 - val_mse: 0.0277
Epoch 275/300
55/55 - 40s - loss: 0.0084 - mse: 0.0280 - val_loss: 0.0169 - val_mse: 0.0294
Epoch 276/300
55/55 - 40s - loss: 0.0094 - mse: 0.0283 - val_loss: 0.0051 - val_mse: 0.0276
Epoch 277/300
55/55 - 40s - loss: 0.0075 - mse: 0.0280 - val_loss: 0.0050 - val_mse: 0.0273
Epoch 278/300
55/55 - 41s - loss: 0.0107 - mse: 0.0284 - val_loss: 0.0059 - val_mse: 0.0270
Epoch 279/300
55/55 - 41s - loss: 0.0070 - mse: 0.0277 - val_loss: 0.0051 - val_mse: 0.0279
Epoch 280/300
55/55 - 40s - loss: 0.0120 - mse: 0.0285 - val_loss: 0.0057 - val_mse: 0.0270
Ep